### 导入

In [1]:
from btree import *
import trans,time,math,transm

class BTree(BTree):
    @staticmethod
    def random_binary_tree(depth,max_value=30):
        """函数重定义
        生成深度为 depth+4 层随机树，叶子取值范围为 [0,max_value]
        1. 0-3 层完全展开
        2. 4层开始，每层至少保留一个叶子节点
        """
        t = BTree(0) # 初始树
        # 补四层节点
        for i in range(4): 
            last_layer = t.last_layer # 最后一层
            for leaf in last_layer:
                BTree.add_left_right_to_node(leaf)
            t._max_depth += 1 # 深度增加
        # 随机生成树
        for i in range(depth-1):
            last_layer = t.last_layer # 最后一层
            split_num = randint(1,len(last_layer)-1) # 随机数目
            split_leaves = sample(last_layer,split_num) # 随机位置
            for leaf in split_leaves:
                BTree.add_left_right_to_node(leaf)
            t._max_depth += 1 # 深度增加
        for leaf in t.leaves: # 叶子节点随机值
            leaf.value = randint(0,max_value)
        return t

def random_nonleaves_seq(n,skip=0):
    """函数重定义 ！！！
    随机生成 n+4 层非叶节点序列
    1. 0-3层完全展开
    2. 4层开始，每层至少保留一个叶节点
    """
    n += 4
    seq = [1,2,4,8]
    while len(seq)!=n-1:
        total = seq[-1] * 2
        seq.append(randint(1,total-1))
    return seq+[0]

### 节点转树

In [ ]:
# 新增函数
class BTree(BTree):
    @staticmethod
    def node_to_list(tree,depth):
        """节点树 -> 列表"""
        positions = [None for i in range(2**(depth+1)-1)]
        for node in tree:
            # 位置平移公式
            dep = node.depth - tree.depth
            pos = node.position - (tree.position-1)*(2**dep)
            positions[pos-1]=node.value
        return positions

### 测试代码 ###
# 初始化
main_tree = BTree.random_binary_tree(7) # 随机 11 层树
subtrees = [node for node in main_tree if node.depth==3] # 取第3层，得八棵树
positions = [BTree.node_to_list(tree,7) for tree in subtrees] # 八棵树转列表

# 测试
sub_trees_by_pos = [BTree.list_to_tree(pos) for pos in positions] # 重新转树
for tree1,tree2 in zip(sub_trees_by_pos,subtrees):
    assert str(tree1)==str(tree2),"重新得到的树不一样！"
print("函数测试通过")

### 测试-旧函数

存在错误，这部分讨论没有意义

In [ ]:
for i in range(10): # 测试 10 棵随机初始树
    tree = BTree.random_binary_tree(7,200) # 初始树：前三层全展开
    positions = BTree.tree_to_positions(tree)
    start_time = time.time()
    single_max_time = 0
    for j in range(50):
        t = time.time()
        leaves = random_leaves_seq(7) # 叶子节点：前四层为0
        trans.ans = math.inf
        trans.dfs(positions,leaves , 0, 0)
        single_time = time.time()-t
        if single_time>single_max_time:
            single_max_time = single_time
        _,min_cost = main(positions,leaves)
        print("\r位置",j,"总用时 %.3fs"%(time.time()-start_time),min_cost==trans.ans,end="")
    print("\nresult",trans.ans,"平均用时 %.4fs"%((time.time()-start_time)/50),
          "最高用时 %.3fs\n"%single_max_time)

In [ ]:
save_vari(tree,"tree.data")
depth_3_nodes = [node for node in tree if node.depth==3]

In [ ]:
for node in depth_3_nodes:
    print(node)

### 测试-新函数

In [ ]:
for i in range(8):
    tree = BTree.random_binary_tree(7,200) # 随机 7 层树
    subtrees = [node for node in tree if node.depth==3] # 节点子树
    positions = [BTree.node_to_positions(tree,7) for tree in subtrees] # 转列表类型
    time_start=time.time()
    for i in range(100):
        tmp_tar = random_leaves_seq(7)[3:]
        transm.test(positions, tmp_tar)
    time_end = time.time()
    print((time_end - time_start)/100) 

### 方法对比-发现错误

In [2]:
for i in range(8):
    ### 初始化 ###
    main_tree = BTree.random_binary_tree(7,200) # 随机 7 层树
    subtrees = [node for node in main_tree if node.depth==3] # 节点子树
    
    # 旧方法：单个列表
    old_positions = BTree.tree_to_list(main_tree)
    leaves = random_leaves_seq(7)
    
    # 新方法：拆 8 个列表
    positions = [BTree.node_to_list(tree,7) for tree in subtrees] 
    tmp_tar = leaves[3:]
    print(leaves)
    
    ### 测试开始 ###
    time_start=time.time()
    ans = transm.test(positions, tmp_tar)
    time_end = time.time()
    print("新法用时 %.3f s"%(time_end - time_start),"计算结果为 %d"%ans)
    
    time_start=time.time()
    optimal,min_cost = main(old_positions,leaves)
    time_end = time.time()
    print("我的用时 %.3f s"%(time_end - time_start),"计算结果为 %d\n"%min_cost)
    
    assert ans == min_cost,"计算结果不匹配"

[0, 1, 1, 1, 1, 1, 2]
新法用时 0.013 s 计算结果为 1350
我的用时 0.002 s 计算结果为 2032



AssertionError: 计算结果不匹配

In [4]:
for node,pos in zip(subtrees,positions):
    print(BTree.position_tree(node))
    print(node)
    print("----")
#     assert str(node)==str(BTree.list_to_tree(pos)),"出错"


           ____8
          /     \
     ____16      17
    /      \
  _32       33
 /   \
64    65


          ____0_
         /      \
     ___0_      141
    /     \
  _0      156
 /  \
76   72

----

     ____9
    /     \
  _18      19
 /   \
36    37


      ____0
     /     \
   _0_      68
  /   \
117   176

----

  _10
 /   \
20    21


   _0_
  /   \
181   127

----

  _11
 /   \
22    23


   _0
  /  \
148   83

----

                                   ____________________________________________________12
                                  /                                                      \
                     ____________24_____________________________________                  25
                    /                                                   \
              _____48____                                           _____49____
             /           \                                         /           \
       _____96_          _97_                             

### 错误数据

In [6]:
### 初始化 ###
main_tree = read_vari("error.data")
subtrees = [node for node in main_tree if node.depth==3] # 节点子树

# 旧方法：单个列表
old_positions = BTree.tree_to_list(main_tree)
leaves = [0, 0, 0, 0, 12, 7, 1, 1, 1, 1, 2]

# 新方法：拆 8 个列表
positions = [BTree.node_to_list(node,7) for node in subtrees] 
tmp_tar = leaves[3:]


ans = transm.test(positions, tmp_tar)
print("新法结果为 %d"%ans)

optimal,min_cost = main(old_positions,leaves)
print("我的结果",min_cost)
assert ans  == min_cost,"计算结果不匹配"

新法结果为 2380
我的结果 2083


AssertionError: 计算结果不匹配

In [10]:
# 获取操作
sep,com = get_operations(main_tree,optimal)
print("拆分位置",sep,"合并位置",com)

# 检查叶子节点
leaf_seq = [0]*11
for leaf in optimal.leaves:
    leaf_seq[leaf.depth] += 1
print("叶子节点序列是否匹配：",leaf_seq == leaves)

拆分位置 set() 合并位置 {108, 16, 19, 437, 25, 29, 30, 31}
叶子节点序列是否匹配： True
